In [ ]:
!pip install ultralytics
!pip install pyngrok

In [2]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!ngrok config add-authtoken YOUR TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [4]:
from flask import Flask, request, render_template_string, send_file
from ultralytics import YOLO
import cv2
import os
import csv
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)

# Load models
detection_model = YOLO('/kaggle/input/face-detection/tensorflow2/default/1/face_detection_model.pt')
classification_model_v8 = YOLO('/kaggle/input/classification-model-for-attendence/tensorflow2/default/1/classification_model_attendence_yolo11.pt')

# Temporary directory to store cropped faces
TEMP_DIR = "/kaggle/working/temp_faces"
os.makedirs(TEMP_DIR, exist_ok=True)

# CSV file to save detected names
CSV_FILE = '/kaggle/working/detected_names.csv'

@app.route('/')
def index():
    return """
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Flask App for Detection and Classification</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                background: #121212;
                color: #f5f5f5;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            h1 {
                color: #00bcd4;
                font-size: 3em;
                margin-bottom: 20px;
            }

            form {
                background-color: #333;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
            }

            label {
                font-size: 1.1em;
                color: #ddd;
            }

            input[type="file"] {
                margin: 10px 0;
                padding: 10px;
                border: none;
                border-radius: 5px;
                background-color: #444;
                color: #fff;
            }

            button {
                padding: 10px 20px;
                background-color: #00bcd4;
                color: white;
                font-size: 1.2em;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                transition: background-color 0.3s ease;
            }

            button:hover {
                background-color: #0097a7;
            }

            a {
                color: #00bcd4;
                text-decoration: none;
                font-size: 1.2em;
                margin-top: 20px;
                display: inline-block;
                transition: color 0.3s ease;
            }

            a:hover {
                color: #0097a7;
            }

            .container {
                text-align: center;
            }

            .names-list {
                list-style-type: none;
                padding: 0;
            }

            .names-list li {
                font-size: 1.4em;
                background: #444;
                margin: 5px 0;
                padding: 10px;
                border-radius: 5px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Face Detection & Classification</h1>
            <form action="/process_image" method="post" enctype="multipart/form-data">
                <label for="image">Upload Image:</label>
                <input type="file" name="image" accept="image/*">
                <button type="submit">Submit</button>
            </form>
        </div>
    </body>
    </html>
    """

@app.route('/process_image', methods=['POST'])
def process_image():
    file = request.files['image']
    if not file:
        return "<h1>Error: No file uploaded!</h1>"

    # Save the uploaded image
    input_path = os.path.join(TEMP_DIR, file.filename)
    file.save(input_path)

    # Run detection model
    results = detection_model.predict(source=input_path, conf=0.6)
    image = cv2.imread(input_path)
    annotations = results[0].boxes

    detected_names = []

    for i, box in enumerate(annotations):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped_face = image[y1:y2, x1:x2]
        cropped_face_path = os.path.join(TEMP_DIR, f'cropped_face_{i}.jpg')
        cv2.imwrite(cropped_face_path, cropped_face)

        # Run classification model
        classify_results = classification_model_v8.predict(source=cropped_face_path, imgsz=128)
        for result in classify_results:
            predicted_class = result.names[result.probs.top1]
            detected_names.append(predicted_class)

    # Save the detected names to CSV
    with open(CSV_FILE, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Detected Name'])
        for name in detected_names:
            writer.writerow([name])

    # Render names in an HTML template
    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Detected Names</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                background: #121212;
                color: #f5f5f5;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            h1 {
                color: #00bcd4;
                font-size: 3em;
                margin-bottom: 20px;
            }

            .names-list {
                list-style-type: none;
                padding: 0;
                margin-bottom: 30px;
            }

            .names-list li {
                font-size: 1.4em;
                background: #444;
                margin: 5px 0;
                padding: 10px;
                border-radius: 5px;
            }

            a {
                color: #00bcd4;
                text-decoration: none;
                font-size: 1.2em;
                margin-top: 20px;
                display: inline-block;
                transition: color 0.3s ease;
            }

            a:hover {
                color: #0097a7;
            }
        </style>
    </head>
    <body>
        <h1>Detected Names</h1>
        <ul class="names-list">
            {% for name in names %}
                <li>{{ name }}</li>
            {% endfor %}
        </ul>
        <a href="/">Upload Another Image</a><br>
        <a href="/download_csv">Download CSV</a>
    </body>
    </html>
    """

    return render_template_string(html_template, names=detected_names)

@app.route('/download_csv')
def download_csv():
    # Send the CSV file as an attachment
    return send_file(CSV_FILE, as_attachment=True)

if __name__ == '__main__':
    # Set up ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f"Your app is running on {public_url}")

    app.run(debug=True, use_reloader=False)


Your app is running on NgrokTunnel: "https://cefd-34-147-110-53.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


In [ ]:
from flask import Flask, request, render_template_string, send_file
from ultralytics import YOLO
import cv2
import os
import csv
from pyngrok import ngrok
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# Load models
detection_model = YOLO('/kaggle/input/face-detection/tensorflow2/default/1/face_detection_model.pt')
classification_model_v8 = YOLO('/kaggle/input/classification-model-for-attendence/tensorflow2/default/1/classification_model_attendence_yolo11.pt')

# Temporary directory to store cropped faces
TEMP_DIR = "/kaggle/working/temp_faces"
os.makedirs(TEMP_DIR, exist_ok=True)

# CSV file to save detected names
CSV_FILE = '/kaggle/working/detected_names.csv'

@app.route('/')
def index():
    return """
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Flask App for Detection and Classification</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                background: #121212;
                color: #f5f5f5;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            h1 {
                color: #00bcd4;
                font-size: 3em;
                margin-bottom: 20px;
            }

            form {
                background-color: #333;
                padding: 20px;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
            }

            label {
                font-size: 1.1em;
                color: #ddd;
            }

            input[type="file"] {
                margin: 10px 0;
                padding: 10px;
                border: none;
                border-radius: 5px;
                background-color: #444;
                color: #fff;
            }

            button {
                padding: 10px 20px;
                background-color: #00bcd4;
                color: white;
                font-size: 1.2em;
                border: none;
                border-radius: 5px;
                cursor: pointer;
                transition: background-color 0.3s ease;
            }

            button:hover {
                background-color: #0097a7;
            }

            a {
                color: #00bcd4;
                text-decoration: none;
                font-size: 1.2em;
                margin-top: 20px;
                display: inline-block;
                transition: color 0.3s ease;
            }

            a:hover {
                color: #0097a7;
            }

            .container {
                text-align: center;
            }

            .names-list {
                list-style-type: none;
                padding: 0;
            }

            .names-list li {
                font-size: 1.4em;
                background: #444;
                margin: 5px 0;
                padding: 10px;
                border-radius: 5px;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <h1>Face Detection & Classification</h1>
            <form action="/process_image" method="post" enctype="multipart/form-data">
                <label for="image">Upload Image:</label>
                <input type="file" name="image" accept="image/*">
                <button type="submit">Submit</button>
            </form>
        </div>
    </body>
    </html>
    """

@app.route('/process_image', methods=['POST'])
def process_image():
    file = request.files['image']
    if not file:
        return "<h1>Error: No file uploaded!</h1>"

    # Save the uploaded image
    input_path = os.path.join(TEMP_DIR, file.filename)
    file.save(input_path)

    # Run detection model
    results = detection_model.predict(source=input_path, conf=0.6)
    image = cv2.imread(input_path)
    annotations = results[0].boxes

    detected_names = []
    prediction_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    for i, box in enumerate(annotations):
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cropped_face = image[y1:y2, x1:x2]
        cropped_face_path = os.path.join(TEMP_DIR, f'cropped_face_{i}.jpg')
        cv2.imwrite(cropped_face_path, cropped_face)

        # Run classification model
        classify_results = classification_model_v8.predict(source=cropped_face_path, imgsz=128)
        for result in classify_results:
            predicted_class = result.names[result.probs.top1]
            detected_names.append(predicted_class)

    # Save the detected names to CSV
    with open(CSV_FILE, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Detected Name', 'Prediction Time'])
        for name in detected_names:
            writer.writerow([name, prediction_time])

    # Render names in a table with prediction time
    html_template = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Detected Names</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                background: #121212;
                color: #f5f5f5;
                margin: 0;
                padding: 0;
                display: flex;
                justify-content: center;
                align-items: center;
                height: 100vh;
                flex-direction: column;
            }

            h1 {
                color: #00bcd4;
                font-size: 3em;
                margin-bottom: 20px;
            }

            table {
                width: 80%;
                margin: 20px 0;
                border-collapse: collapse;
                background-color: #333;
                border-radius: 10px;
            }

            th, td {
                padding: 12px;
                text-align: center;
                border: 1px solid #444;
            }

            th {
                background-color: #00bcd4;
                color: white;
            }

            td {
                background-color: #444;
                color: #f5f5f5;
            }

            a {
                color: #00bcd4;
                text-decoration: none;
                font-size: 1.2em;
                margin-top: 20px;
                display: inline-block;
                transition: color 0.3s ease;
            }

            a:hover {
                color: #0097a7;
            }
        </style>
    </head>
    <body>
        <h1>Detected Names and Prediction Time</h1>
        <table>
            <thead>
                <tr>
                    <th>Detected Name</th>
                    <th>Prediction Time</th>
                </tr>
            </thead>
            <tbody>
                {% for name in names %}
                <tr>
                    <td>{{ name }}</td>
                    <td>{{ time }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        <a href="/">Upload Another Image</a><br>
        <a href="/download_csv">Download CSV</a>
    </body>
    </html>
    """

    return render_template_string(html_template, names=detected_names, time=prediction_time)

@app.route('/download_csv')
def download_csv():
    # Send the CSV file as an attachment
    return send_file(CSV_FILE, as_attachment=True)

if __name__ == '__main__':
    # Set up ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f"Your app is running on {public_url}")

    app.run(debug=True, use_reloader=False)

Your app is running on NgrokTunnel: "https://ce6b-34-147-110-53.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: on

image 1/1 /kaggle/working/temp_faces/testimage.jpg: 480x640 7 WIDER FACE - v1 2023-10-19 4-18pms, 31.4ms
Speed: 2.0ms preprocess, 31.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /kaggle/working/temp_faces/cropped_face_0.jpg: 128x128 abdullah saaz 0.27, karrar 0.25, atta ur rehman 0.13, aligohar 0.10, asad 0.08, 4.2ms
Speed: 2.1ms preprocess, 4.2ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

image 1/1 /kaggle/working/temp_faces/cropped_face_1.jpg: 128x128 mahad 0.50, atta ur rehman 0.18, haris 0.09, qasim 0.07, kashif 0.06, 4.4ms
Speed: 1.9ms preprocess, 4.4ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 128)

image 1/1 /kaggle/working/temp_faces/cropped_face_2.jpg: 128x128 mahad 0.82, maqsood 0.10, mahaz 0.03, ali raza 0.02, hassan wadood 0.02, 4.4ms
Speed: 1.